In [2]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import plotly.express as px


In [3]:


df = pd.read_csv('../data/activities/activity.csv', encoding='utf-8',sep = ",", header=0, skiprows=[1])
df


,Duration,Distance,OriginalPace,HeartRate,Cadence,PowerOriginal,CalculatedPace,CalculatedStrideLength,CalculatedAerobicEfficiencyPace,CalculatedAerobicEfficiencyPower,CalculatedEfficiencyIndex,Unnamed: 11
0,1,0.00000,NaN,94,0,0,NaN,0,NaN,NaN,NaN,NaN
1,1,0.00000,NaN,94,0,0,NaN,0,NaN,NaN,NaN,NaN
2,1,0.00000,NaN,96,0,0,NaN,0,NaN,NaN,NaN,NaN
3,1,0.00000,NaN,96,0,0,NaN,0,NaN,NaN,NaN,NaN
4,1,0.00000,NaN,97,0,0,NaN,0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1798,1,11.94671,319.0,132,51,32,NaN,184,1.12,0.24,5.878,NaN
1799,1,11.95022,337.0,131,40,18,NaN,223,1.06,0.14,9.891,NaN
1800,1,11.95264,365.0,130,30,5,NaN,274,0.99,0.04,32.877,NaN
1801,1,11.95449,384.0,130,22,0,NaN,355,0.94,NaN,NaN,NaN


In [4]:
power_data = df['PowerOriginal'][10:300] 

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=power_data.index,
    y=power_data,
    mode='lines+markers',
    name='Power'
))






In [5]:
# Auflösung 10W
resolution = 200

last_index = 0
series_list = []

max_power = power_data.max()

power_data_cut = power_data[power_data>max_power-resolution]

for index, value in power_data_cut.items():
    if last_index and last_index == index - 1:
        series_list[-1] += 1
    else:
        series_list.append(1)
    last_index = index

print(series_list)


[30, 13, 4, 3]


In [17]:
def maxtime_spend_in_power(power_series, power):
    last_index = 0
    series_list = [0]

    max_power = power_data.max()
    power_data_cut = power_data[power_data>max_power-power]

    for index, value in power_data_cut.items():
        if last_index and last_index == index - 1:
            series_list[-1] += 1
        else:
            series_list.append(1)
        last_index = index

    return max(series_list)

def sort_max_time(resolution):
    # resolution = 1
    data = []
    for p in range(0, power_data.max(), resolution):
        data.append([power_data.max()-p, maxtime_spend_in_power(power_data, p)])
        #print(f"Max time spend in {power_data.max()-p}W: {maxtime_spend_in_power(power_data, p)}")

    df = pd.DataFrame(data, columns=['Leistung', 'Maximale Zeit'])
    return df

sort_max_time(1)

,Leistung,Maximale Zeit
0,433,0
1,432,1
2,431,1
3,430,1
4,429,2
...,...,...
428,5,595
429,4,596
430,3,596
431,2,596


In [18]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    y=df["Leistung"],
    x=df["Maximale Zeit"],
    mode='lines+markers',
    name='Power Curve'
))

In [8]:
# Funktion zur Berechnung der maximalen zusammenhängenden Zeitspanne über einem gegebenen Leistungswert
def maxtime_spend_in_power(power_series, power_threshold):
    last_index = None
    series_list = []

    # Filtern der Leistungswerte über dem Schwellenwert
    power_data_cut = power_series[power_series > power_threshold]

    for index in power_data_cut.index:
        if last_index == index - 1:
            series_list[-1] += 1
        else:
            series_list.append(1)
        last_index = index

    # Gibt die maximale Zeitspanne zurück
    return max(series_list) if series_list else 0

# CSV-Datei einlesen
df = pd.read_csv('../data/activities/activity.csv', encoding='utf-8', sep=",", header=0, skiprows=[1])

# Annahme: Die Spalte mit den Leistungsdaten heißt 'PowerOriginal'
power_data = df['PowerOriginal']

# Schwellenwert für die Leistung
power_threshold = 420

# Berechnung der längsten Zeitspanne über 400 Watt
max_time_over_threshold = maxtime_spend_in_power(power_data, power_threshold)
print(maxtime_spend_in_power(power_data, power_threshold))
print(f"Die längste Zeitspanne über {power_threshold} Watt beträgt {max_time_over_threshold} Sekunden.")

2
Die längste Zeitspanne über 420 Watt beträgt 2 Sekunden.


In [9]:
import pandas as pd

def maxtime_spend_in_power(power_series, power_threshold):
    last_index = None
    series_list = []

    # Filtern der Leistungswerte über dem Schwellenwert
    power_data_cut = power_series[power_series > power_threshold]

    for index in power_data_cut.index:
        if last_index is not None and last_index == index - 1:
            series_list[-1] += 1
        else:
            series_list.append(1)
        last_index = index

    # Gibt die maximale Zeitspanne zurück
    return max(series_list) if series_list else 0

# CSV-Datei einlesen
df = pd.read_csv('../data/activities/activity.csv', encoding='utf-8', sep=",", header=0, skiprows=[1])

# Annahme: Die Spalte mit den Leistungsdaten heißt 'PowerOriginal'
power_data = df['PowerOriginal']

# Liste für die Ergebnisse
results = []

# Schrittweite für die Schwellenwerte
step_size = 1

# Maximum der Leistungsdaten
max_power = power_data.max()

# Schleife von der höchsten Leistung zum niedrigsten Wert in Schritten von step_size
for power_threshold in range(int(max_power), -1, -step_size):
    max_time_over_threshold = maxtime_spend_in_power(power_data, power_threshold)
    results.append([power_threshold, max_time_over_threshold])

# Ergebnisse in ein DataFrame umwandeln
results_df = pd.DataFrame(results, columns=['Leistung', 'Maximale Zeit'])

# Ergebnisse anzeigen
results_df


,Leistung,Maximale Zeit
0,433,0
1,432,1
2,431,1
3,430,1
4,429,2
...,...,...
429,4,596
430,3,596
431,2,596
432,1,596


In [11]:
# Plotly-Diagramm erstellen
fig = px.line(results_df, x='Maximale Zeit', y='Leistung', title='Maximale Zeit über Leistungswerten', labels={'Leistung': 'Leistung (Watt)', 'Maximale Zeit': 'Maximale Zeit (Sekunden)'})

# Diagramm anzeigen
fig.show()